In [136]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import ast

In [137]:
molecules = pd.read_csv('../datas/molecules.csv', names=['id', 'none', 'pubchem_id', 'common_name', 'flavor_profile'], index_col='pubchem_id')
molecules.drop(['id', 'none'], axis=1, inplace=True)
flavor_db = pd.read_csv('../datas/flavordb.csv', names=['id', 'entry_id', 'alias', 'synonyms', 'scientific_name', 'category', 'molecules'], index_col='entry_id')
flavor_db.drop(['id'], axis=1, inplace=True)

In [138]:
# Make all list of all flavors
all_flavors = molecules['flavor_profile'].apply(lambda x: x.replace("'", "").strip('{}').split(', ')).tolist()
unique_flavors = set([flavor for flavors in all_flavors for flavor in flavors])
print(f"{len(unique_flavors)}種類")
print(unique_flavors)

600種類
{'', 'peanut', 'old wood', 'maple', 'syrup', 'hawthorn', 'must', 'aromatic', 'black tea', 'gooseberry', 'saffron', 'ripe', 'storax', 'raspberry', 'rubbery', 'rose dried', 'mousy', 'labdanum', 'malt', 'gardenia', 'mango', 'medicine', 'resinous', 'slightly waxy', 'cauliflower', 'green tea', 'resin', 'dust', 'musky', 'toluene', 'leather', 'paper', 'petitgrain', 'urine', 'earth', 'vanillin', 'harsh', 'pear', 'cat-urine', 'menthol', 'very mild', 'shellfish', 'weak spice', 'melon', 'peppery', 'terpenic', 'fish', 'very strong', 'juicy', 'paint', 'coconut', 'tomato', 'chamomile', 'smoke', 'flat', 'beef', 'plastic', 'mimosa', 'leaf', 'rummy', 'decomposing cabbage', 'leafy', 'guaiacwood', 'opoponax', 'cream', 'popcorn', 'clover', 'biscuit', 'parsley', 'almond', 'rotting', 'unripe fruit', 'sweaty', 'patchouli', 'citrus peel', 'almond shell', 'caramel', 'rose acetate', 'solvent', 'nutty', 'muguet', 'unripe plum', 'cinnamon', 'toffee', 'alkaline', 'marshmallow', 'orange flower', 'fragrant', '

In [139]:
molecules.tail(3)

,common_name,flavor_profile
pubchem_id,,
10340,Sodium Carbonate,odorless
24856,Potassium alum,odorless
24403,Tetrasodium Pyrophosphate,odorless


In [140]:
# Make all list of all categories 
unique_categories = set([cate for cate in flavor_db['category']])
print(f"{len(unique_categories)}種類")
print(unique_categories)

35種類
{'essential oil', 'vegetable stem', 'maize', 'dish', 'vegetable fruit', 'nut', 'vegetable root', 'cabbage', 'plant derivative', 'berry', 'additive', 'herb', 'fish', 'dairy', 'meat', 'bakery', 'cereal', 'beverage', 'seafood', 'vegetable', 'beverage caffeinated', 'flower', 'legume', 'seed', 'gourd', 'fruit', 'fruit essence', 'category', 'fungus', 'fruit citrus', 'plant', 'vegetable tuber', 'fruit-berry', 'beverage alcoholic', 'spice'}


In [141]:
flavor_db.tail(5)

,alias,synonyms,scientific_name,category,molecules
entry_id,,,,,
970,saskatoon berry,"{'alder-leaf shadbush', 'western serviceberry'...",amelanchier,berry,"{644104, 527, 8723, 31260, 15394, 6184, 439341..."
971,nanking cherry,"{'ando cherry', 'downy cherry', 'shanghai cher...",prunus cerasus,berry,"{644104, 527, 8723, 31260, 15394, 6184, 439341..."
972,japanese pumpkin,"{'kabocha', 'japanese pumpkin'}",winter squash,fruit,"{644104, 527, 8723, 31260, 15394, 6184, 439341..."
977,guinea hen,"{'pet speckled hen', 'original fowl', 'guinea ...",galliformes,meat,"{644104, 1130}"
978,cucurbita,{'cucurbita'},cucurbita,gourd,"{644104, 527, 8723, 31260, 15394, 6184, 65064,..."


In [142]:
entry_molecules = flavor_db[['molecules']].apply(lambda x: x.replace("{", "[").replace("}","]"))
entry_molecules
#entry_molecules = flavor_db.explode('molecules')
#print(entry_molecules)

,molecules
entry_id,
entity id,molecules
1,"{27457, 7976, 31252, 26808, 22201, 26331}"
2,"{1031, 1032, 644104, 527, 8723, 31260, 15394, ..."
3,"{644104, 7824, 643731, 8468, 1049, 5372954, 80..."
4,"{6915, 5365891, 12170, 8082, 31251, 7958, 1049..."
...,...
970,"{644104, 527, 8723, 31260, 15394, 6184, 439341..."
971,"{644104, 527, 8723, 31260, 15394, 6184, 439341..."
972,"{644104, 527, 8723, 31260, 15394, 6184, 439341..."


In [143]:
# 共起関係を計算
co_occurrence = flavor_db.T.dot(flavor_db)

# ネットワークの作成
G = nx.Graph()

# ノードとエッジを追加
for ingredient in co_occurrence.columns:
    G.add_node(ingredient)

for i, j in itertools.combinations(co_occurrence.columns, 2):
    weight = co_occurrence.at[i, j]
    if weight > 0:  # 共起頻度が0より大きい場合のみエッジを追加
        G.add_edge(i, j, weight=weight)

# ネットワークの描画
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, font_weight='bold')

# エッジのラベル（重み）を表示
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

plt.show()

TypeError: can't multiply sequence by non-int of type 'str'